In [1]:
%load_ext autoreload


In [8]:
from random import random, randint
import numpy as np
from copy import deepcopy

In [3]:
nlines = 31
ncols = 7
k = 8

def cria_linha(ncols): #ncols = 7
    l=[]
    i=0
    t=0
    while i<ncols:
        n=randint(0,1) # mudar probabilidade!!!!!!!! assim nao funciona
        l.append(n)
        t+=n
        i+=1
    if t:#checa se pelo menos um membro percorreu o percurso
        return l
    else:
        return cria_linha(ncols)
   
#cria matriz [30][7]
def cria_matriz(nlines, ncols): # nlines=30, ncols = 8
    i=0
    m=[]
    while i<nlines:
       m.append(cria_linha(ncols))
       i+=1
    return m


def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    vivo = randint(0, ncols-1)
    for i in range(ncols):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < k:
         n = randint(0,nlines-1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0,ncols-1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, nlines-1)
        while m[k][n]== 0:
            k = randint(0, nlines-1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 


In [4]:
nlines = 31
ncols = 7
k = 8
#checa se alguem foi usado mais que k vezes
#checa se alguem foi usado mais que k vezes
def checa_k_vivo(*m):
    alive = 0
    for i in range(len(m[0])):
        t=0
        for j in range(len(m)):
            t+=(m[j][i])
            # j+=1z
        alive += t
        if t>k:
            return i #retorna indice do que apareceu mais de k vezes
    if alive >= k*len(m[0]):
        return -2
            
        # i+=1
    return -1


def checa_vazio(*m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1
    
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    # x = (person, time)
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[0][line][col] != y[0][line][col]:
                tuplas.append((line, col))
    casas_sorteadas = []
    n = min(5, len(tuplas))
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(num)
    childs = [x, y]
    bits = np.zeros(n)
    bits[0] = 1
    m = deepcopy(x[0])
    l = pow(2,n)
    for i in range(l-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[0][a][b]
        um = np.zeros(n)
        um[0] = 1
        bits = soma_bits(bits, um, n)
        if checa_k_vivo(*matrix) == -1 and checa_vazio(*matrix) == -1:
            time = custo_tempo(etapa_dif, personagem_agilidade, matrix)
            childs.append((matrix, time))
    return childs

In [10]:

etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]
nlines = 31
ncols = 7
k = 8
# custo do tempo a cada etapa: dificuldade/ agilidade na etapa
def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix)



def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = list()
    for person in population:
        times.append(person[1])
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] # (person, time)
    
def best_individual(population):
    population.sort(key=lambda y: y[1]) #sort by time
    best_time = population[0][1]
    best_person = population[0][0]
    return best_time, best_person

def die(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1])
    for i in range(1,excess+1):
        population.pop(tam-i)
    # return population

def repetition(population, potential_child):
    for child in population:
        if np.array_equal(child[0], potential_child[0]):
            return 1
    return 0

def genetic_algorithm(population): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)
    count = 0
    while count < 50:
    ## tamanho pop
    # ideia tirar caras ruins
        new_population = []
        # evaluate best individual...
        for i in range(200):
            x = random_selection(population) # (person, time)
            y = random_selection(population)
            while np.array_equal(x[0],y[0]):
                y = random_selection(population)
            childs = reproduce1(x,y)
            for child in childs:
                if repetition(new_population, child) == 0:
                    new_population.append(child)
            # p = random()
            # if (p < 0.01):
            #     child = mutate(child)
        population = new_population
        print('new pop', len(population))
        if len(population) > 3000:
            die(population, 3000)
            print('new pop after massive deaths', len(population))
        
        time, ind = best_individual(population)
        if time < best_time:
            best_time = time
            fit_ind = ind
            print('time', time)
            print('best_time', best_time)
        else:
            count += 1
            
    return fit_ind

In [11]:
population = list()
count = 0
while count < 100:
    potential_person = cria_novo_matrix(nlines, ncols, k)
    if repetition(population, potential_person) == 0:
        tempo = custo_tempo(etapa_dif, personagem_agilidade, potential_person)
        population.append((potential_person, tempo))
        count +=1

In [12]:
len(population)

100

In [13]:
genetic_algorithm(population)

best_time 2371.209406637327
new pop 1314
time 2358.4458854634686
best_time 2358.4458854634686
new pop 1771
new pop 2042
time 2344.285029229083
best_time 2344.285029229083
new pop 2288
new pop 2265
time 2312.600728176815
best_time 2312.600728176815
new pop 2640
new pop 2528
time 2271.1364424625294
best_time 2271.1364424625294
new pop 2641
new pop 2800
new pop 2905
new pop 3027
new pop after massive deaths 3000
new pop 3088
new pop after massive deaths 3000
new pop 2707
time 2266.8120864287744
best_time 2266.8120864287744
new pop 3249
new pop after massive deaths 3000
new pop 2882
time 2222.3407570178656
best_time 2222.3407570178656
new pop 3097
new pop after massive deaths 3000
new pop 2996
new pop 3420
new pop after massive deaths 3000
new pop 3205
new pop after massive deaths 3000
time 2189.9919187119067
best_time 2189.9919187119067
new pop 3200
new pop after massive deaths 3000
time 2170.9086794189
best_time 2170.9086794189
new pop 3425
new pop after massive deaths 3000
new pop 3390


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0.],
 

In [ ]:
time, ind = best_individual(population)

In [ ]:
time

In [ ]:
min(4,1)

In [ ]:
x = population[0]
y= population[1]

In [ ]:
x

In [ ]:
y

In [ ]:
childs= reproduce1(x,y,2)

In [ ]:
len(childs)

In [ ]:
np.array_equal(childs[1], childs[2])

In [ ]:
a = genetic_algorithm(population, 40)

In [ ]:

def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[line][col] != y[line][col]:
                tuplas.append((line, col))

    casas_sorteadas = []
    n=4
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(k)
    
    ## np.array
    childs = [x, y]
    bits = np.array([0,0,0,0])
    m = deepcopy(x)
    n = pow(2,n)
    for i in range(n-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[a][b]
        um = np.array([1,0,0,0])
        bits = soma_bits(bits, um, n)
        if checa_k(matrix, k) == -1 and checa_vazio(matrix) == -1:
            childs.append(matrix)
    return childs

In [ ]:
from random import random

In [ ]:
m = np.matrix([[0,1], [1,1], [0,0]])
checa_vazio(*m)

In [ ]:
lista =[(1,2), (3,3)]

In [ ]:
n = np.zeros((4, 5))

In [ ]:
lista

In [ ]:
lista.append((3,4))

In [ ]:
lista

In [ ]:
m = [[1,2],[3,4]]
n = deepcopy(m)

lista = []
lista.append(n)
m[0][0] = 7


In [ ]:
n = deepcopy(m)
lista.append(n)

n[1][1] = 8

In [ ]:
from copy import deepcopy

In [ ]:
lista

In [ ]:
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits


In [ ]:
bits(16)

In [ ]:
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

In [ ]:
soma_bits(a,b,3)

In [ ]:
a = np.array([1, 0, 1])
b = np.array([1,0,1])

In [ ]:
lista + list1

In [ ]:
lista.pop(1)

In [ ]:
lista

In [ ]:
np.append(m, [2])

In [ ]:
import numpy as np

In [ ]:
m = np.zeros((3,3))

In [ ]:
m

In [ ]:
n

In [ ]:
n[0][0] =1

In [ ]:
n = m.copy()

In [ ]:
lista = [[0,2], [1,1]]

In [ ]:
lista[0][1]

In [ ]:
lista[:,2]